In [0]:
##############################################################
#Implementation of the Boneh-Franklin Identity Based Encryption using the Weil Pairing
#Florence Lam
#Accompanies a paper with the same title
#References to Theorems/Definitions/Algorithms/Sections are to that paper
##############################################################


#BasicIdent requires two hash functions (cf. Section 2.4)
#We use this library
from Crypto.Hash import SHA512

#Okay for Hash(ID)
def H1(hash_input):
    hash = SHA512.new()
    str_val = str(hash_input)
    byte_val = str_val.encode()
    hash.update(byte_val)
    hexadecimal = hash.hexdigest()
    return int(hexadecimal, 16)

In [0]:
#Fast modular exponentiation algorithm (needed for F_{p^2})
def fastPowerSmall(g,A):
    a = g
    b = 1
    if g == 0 and A == 0:
        return "Undefined"
    else:
        while A > 0:
            if A%2 == 1:
                b = b*a
            a = a^2
            A = A//2
            print
        return b

In [0]:
#ASCII implementation
def textToInt(w):
    n  = 0
    counter = 0
    #counter will give us the index of each character in the string
    for i in w:
        n = n + ord(i)*(256**counter)
        counter = counter + 1
    return n

def intToText(n):
    str = ""
    while n > 0:
        a0 = n%256
        str = str + chr(a0) #chr undoes ord. ord() inputs character and outputs integer. str inputs integer between 0 and 255 and outputs character.
        n = n//256 #This is the quotient after dividing n by 256
    return str

In [0]:
#First step of Miller's Algorithm
#This defines a rational function g(x,y) on E whose divisor is div(g) = [P] + [Q] - [P+Q] - [O]
#(cf. Theorem 2.2)

#A, B coefficients of E. Use A,B = E.a_invariants()[3], E.a_invariants()[4]
#P = E([xP, yP])

def g(P,Q,x,y,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    if P == E(0) or Q == E(0):
        return "no divisor"
    xP,yP = P[0],P[1]
    xQ,yQ = Q[0],Q[1]
    #Calculate slope of line connecting P and Q
    #JUST check if equal
    if yP == -yQ and xP == xQ:
        slope = +oo #symbol for Infinity
    elif P == Q:
        slope = (3*(xP**2) + A)/(2*yP)
    else:
        slope = (yQ - yP)/(xQ - xP)
    #return the function on E
    if slope == +oo:
        return x - xP
    else:
        return (y - yP - slope*(x - xP))/(x + xP + xQ - slope**2)

In [0]:
#Implementation of Miller's algorithm (Theorem 2.2)

def MillerAlgorithm(P,m,x,y,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    xP,yP = P[0],P[1]
    binary = m.digits(2) #gives number in binary
    n = len(binary) #trying to find what "n" is.
    T = P
    f = 1
    for i in range(n-2,-1,-1): #Stop once i = -1, so last number is 0...range(start, stop, step)
        f = (f**2)*g(T,T,x,y,E)
        T *= 2 #T = 2T
        if binary[i] == 1:
            f = f*g(T,P,x,y,E)
            T += P
    return f #This algorithm returns a value...

In [0]:
#Weil Pairing Implementation (cf. Theorem 2.3)
#Works on general elliptic curve over finite field

def WeilPairing(P,Q,m,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    S = E.random_element()
    while m*S == E(0):
        S = E.random_element() #Pick point S that is not m-torsion. This guarantees that S isn't a linear combination of P and Q.
    xS,yS = S[0],S[1]
    QplusS = Q + S
    f_P_QplusS = MillerAlgorithm(P,m,QplusS[0],QplusS[1],E)
    f_P_S = MillerAlgorithm(P,m,xS,yS,E)
    num = f_P_QplusS/f_P_S

    PminusS = P - S
    f_Q_PminusS = MillerAlgorithm(Q,m,PminusS[0],PminusS[1],E) #This is f_Q(P-S)
    f_Q_minusS = MillerAlgorithm(Q,m,xS,-yS,E) #This is f_Q(-S)
    denom = f_Q_PminusS/f_Q_minusS

    e_m = num/denom
    return e_m

In [0]:
#Implementation of Modified Weil Pairing (cf. Definition 2.8)
#Works on the elliptic curve E given by y^2 = x^3 + 1 over F_p with p a prime congruent to 2 modulo 3

def ModifiedWeilPairing(P,Q,m,E):
    Fp = GF(p)
    R.<x> = Fp[]
    Fp2.<z> = Fp.extension(x^2+x+1)      #Form F_{p^2} by adjoining z = {a nontrivial cube root of 1}
    E_zeta = EllipticCurve(Fp2, [0,1])   #Define E: y^2 = x^3 + 1 over this field
    phiQ = E_zeta([z*Q[0],Q[1]])
    A,B = E_zeta.a_invariants()[3], E_zeta.a_invariants()[4]
    P_zeta = E_zeta([P[0],P[1]])
    S = E_zeta.random_element()
    while m*S == E(0):
        S = E_zeta.random_element()
    xS,yS = S[0],S[1]
    QplusS = phiQ + S
    f_P_QplusS = MillerAlgorithm(P,m,QplusS[0],QplusS[1],E_zeta)
    f_P_S = MillerAlgorithm(P,m,xS,yS,E_zeta)
    num = f_P_QplusS/f_P_S

    PminusS = P_zeta - S #modify
    f_Q_PminusS = MillerAlgorithm(phiQ,m,PminusS[0],PminusS[1],E_zeta) #This is f_Q(P-S)
    f_Q_minusS = MillerAlgorithm(phiQ,m,xS,-yS,E_zeta) #This is f_Q(-S)
    denom = f_Q_PminusS/f_Q_minusS

    e_m = num/denom
    return e_m

In [0]:
#Implementation of BDHGenerator
#THis is `Setup` algorithm in IBE scheme ###2
#(cf. Algorithm 2 in Section 3.1)

def BDHGenerator(k):
    q = random_prime((2^k) - 1, True, lbound=2^(k-1)) #Generates a random k-bit prime. False means using pseudo-primality tests.
    p = q
    l = 1 #need l for `MapToPoint`
    lq = q
    while True:
        l += 1
        lq += q
        p = lq - 1
        if p%3 == 2 and (p+1)%(q^2) != 0 and is_prime(p) == True:
            break
    E = EllipticCurve(GF(p), [0,1]) #p is public b/c the elliptic curve is known
    P = E(0)
    while P == E(0):
        Q = E.random_element()
        while Q == E(0): #make sure P is not O
            Q = E.random_element()
        h = (p+1)//q #This is to make sure P has order q.
        P = h*Q #Order of P is q1
    s = ZZ.random_element(2,q-1) #s is private master key in Z_q*.
    P_pub = s*P
    params = [p, q, l, E, P, P_pub]
    return params, s #Everything except s is `params`/public

In [0]:
#Implementation of MapToPoint
#(cf Algorithm 3 in Section 3.2)

#A brief sketch of how this is used:
#y0 in Fp. It is an output from hash function. Input to the first hash function is ID.
#s is the master key
#private key is d_ID (don't send to others)
#MapToPoint is `Extract` in Boneh-Franklin
#Q_ID is for people to use so they can encrypt. They can just use ID, hash it, and calculate Q_ID by themselves. One way is to make a `public` MapToPoint function that doesn't take in s, and doesn't d_ID
#Admin does this, not the user
#Hash y0 before inputting to MapToPoint. y0 = Hash(ID)
#y0 is an element mod p, need to reduce mod p.

def MapToPoint(y0,params):
    p, q, l, E, P, P_pub = params
    x0 = pow((y0^2) - 1, ((2*p)-1)//3, p) #`pow` is Python's built-in function that does fast power
    Q = E([x0,y0])
    Q_ID = l*Q #l comes from BDHGenerator. It's the integer s.t. p = lq-1
    d_ID = s*Q_ID
    return Q_ID,d_ID

In [0]:
###Encrypt and Decryptimplementation
#(cf. Section 2.4 and end of Section 3.2)

def Encrypt(M,Q_ID,params):
    params = [p, q, l, E, P, P_pub]
    r = ZZ.random_element(2,q-1)
    U = r*P
    g_ID = ModifiedWeilPairing(Q_ID,P_pub,q,E)
    g_ID_to_r = fastPowerSmall(g_ID, r)
    V = M^^H1(g_ID_to_r)
    C = [U,V] #This is the ciphertext
    return C

#^^ is XOR operator in Sage
#d_ID from MapToPoint. It's the private key (I keep it)

def Decrypt(C,d_ID,params):
    params = [p, q, l, E, P, P_pub]
    U,V = C
    weil = ModifiedWeilPairing(d_ID,U,q,E)
    M = V^^H1(weil)
    return M


In [0]:
#########################################
#What follows is an example runthrough of the implementation in 5 steps
#Follows the outline of a general IBE scheme (cf. Section 2.4) implemented using the modified Weil pairing
#The user is encouraged to run the code and experiment
#########################################

In [0]:
#Step 1: Only need to run once
#Boneh-Franklin Page 20 says to pick primes p that are at least 512-bit long for security purposes.
params,s = BDHGenerator(512)
p, q, l, E, P, P_pub = params
print("p = ", p)
print("q = ", q)
print("E = ", E)
print("P = ", P)
print("s = ", s)
print("l = ", l)
print("P_pub = ", P_pub)

In [0]:
#Step 2: Hash. Only need to run once for one ID.
your_ID = "user"

y0 = H1(your_ID)
print("y0 = ", y0)

In [0]:
#Step 3: MapToPoint. Only need to run once.
Q_ID,d_ID = MapToPoint(y0,params)
print("Q_ID = ", Q_ID)
print("d_ID = ", d_ID)

In [0]:
#Step 4: Encrypt
input_message = "Hello! :)"

M = textToInt(input_message)
C = Encrypt(M,Q_ID,params)
U,V = C
print("M = ", M)
print("U = ", U)
print("V = ", V)

In [0]:
#Step 5: Decrypt
M0 = Decrypt(C,d_ID,params)
output_message = intToText(M0)
print("M0 = ", M0)
print("messsage = ", output_message)

In [0]:
################
#Open Cell for the user to test and experiment with any and all functions above
################